In [1]:
from vampyr import vampyr3d as vp
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
from qcelemental.physical_constants.context import PhysicalConstantsContext
%matplotlib inline

# initialize classes and functions used in the notebook

In [2]:
class Cavity(object):
    def __init__(self, cav_coords, radii, width):
        self.r_list = cav_coords # list of cavity centers. Normally, but not always, nucleus coordinates. 
        self.R_list = radii  # list of cavity radii. Normally, but not always, nucleus radii.
        self.sigma = width  # width of the cavity boundary
   

    def __call__(self,  r):
        """
        r is a list of floats of length 3, can be a numpy array as well
        """
        r_vec = np.array(r)
        C = 1.0
        for i, r_i in enumerate(self.r_list):
            r_vec_i = np.array(r_i)
            s_i = np.linalg.norm(r_vec_i - r_vec) - self.R_list[i]
            O_i = (1.0/2.0)*(1 + erf(s_i/self.sigma))
            C_i = 1 - O_i
            C *= 1 - C_i
        C = 1.0 - C
        return C

    
class ShiftFunction():
    def __init__(self, cavity, inside=1.0, outside=2.0):
        
        self.C = cavity
        self.inside = inside # permittivity of free space
        self.outside = outside # permittivity of solvent
    
    def __call__(self):
        raise NotImplementedError()


class LinPermittivity(ShiftFunction):
    def __call__(self, r):
        C_eval = self.C(r)
        permittivity = C_eval*(self.inside - self.outside) + self.outside
        return permittivity


class ExpPermittivity(ShiftFunction):
    def __call__(self, r):
        C_eval = self.C(r)
        permittivity = self.inside*np.exp((np.log((self.inside/self.outside)))*(1.0 - C_eval))
    
        return permittivity


class DHScreeningSq(ShiftFunction):
    def __call__(self, r):
        C_eval = self.C(r)
        return (1-C_eval)*(self.outside**2)


# Analytic nuclear potential
def f_nuc(r):
    R = np.sqrt(r[0]*r[0] + r[1]*r[1] + r[2]*r[2])
    return -1.0 / R


# Analytic guess for solution
def f_phi(r):
    
    R = np.sqrt(r[0]*r[0] + r[1]*r[1] + r[2]*r[2])
    return np.exp(-R*R)


def compute_kappa_out(eps_out, Ionic_strength):
    context = PhysicalConstantsContext(context="CODATA2018")
    kb = context.kb
    e = context.get("elementary charge")
    e_0 = context.e0
    N_a = context.na
    m2au = 1/ context.bohr2m
    T = 298.15

    numerator = e_0*eps_out*kb*T
    denominator = 2.0*(e**2)*N_a*1000.0*Ionic_strength
    
    debye_length = ((numerator/denominator)**(1.0/2.0))*m2au
    kappa = 1.0/debye_length
    return kappa


class Reaction_operator():
    def __init__(self, Solver):
        print("Initializing reaction operator")
        self.Solver = Solver
        
    def setup(self, prec):
        print("Setting up reaction operator")
        self.Solver.setup(prec)

    def trace(self):
        return self.Solver.computeEnergy()

In [3]:

class GPESolver():
    instances = []
    def __init__(self, rho, eps, Poisson_operator, Derivative_operator):
        self._attributes = {
            "Density": rho
        }
        self.Permittivity = eps
        self.P = Poisson_operator
        self.D = Derivative_operator
        self.V_R = rho
        self.instance_index = len(GPESolver.instances)
        GPESolver.instances.append(self)
        
    
    def plotFunction(self, function, left=-1.0, right=1.0, npoints=100, title=""):
        x_plt = np.linspace(left, right, npoints)
        f_plt = [function([x, 0.0, 0.0]) for x in x_plt]
        plt.plot(x_plt, f_plt)
        plt.title(title)
        plt.show()
        
    def computeGamma(self, V_tot):
        return (1.0/(4*np.pi)) * vp.dot( vp.gradient(self.D, self.Permittivity), vp.gradient(self.D,V_tot)) * ( self.Permittivity**(-1))
    
    @property
    def Density(self):
        return self._attributes["Density"] 
    
    @Density.setter
    def Density(self, rho):
        self._attributes["Density"] = rho
    
    @property 
    def V_R(self):
        return self._V_R

    @V_R.setter
    def V_R(self, V_r):
        self._V_R = V_r
    
    def setup(self, prec):
        # compute rho_eff = rho/eps

        rho_eff = self.Density * (self.Permittivity)**(-1)
        
        # compute vacuum potential V_vac = \\int rho(r')/|r-r'| dr'
        V_vac = self.P(self.Density)
        
        # start loop
        update = 1.0
        print("iter\t|V_R norm\t\t|update\t\t|E_r\t\t|E_r update")
        E_r_old = 0.0
        for i in range(100):
            
            if (i==0):
                V_tot = V_vac
            else:
                V_tot = V_vac + self.instances[self.instance_index].V_R
            
            # compute the surface charge distribution gamma= 1/4pi * \\nabla log(eps)\\cdot\\nabla V_tot
            # in the first iteration, V_tot = V_vac else V_tot = V_vac + V_R
            gamma = self.computeGamma(V_tot)

            # solve the generalized poisson equation for V_tot
            V_tot_np1 = self.P(rho_eff + gamma)
            #substract V_vac from V_tot to get V_R
            V_R_np1 = V_tot_np1 - V_vac
            if (i==0):
                dV_R = V_R_np1
                self.instances[self.instance_index].V_R = V_R_np1
            else:
                dV_R = V_R_np1 - self.instances[self.instance_index].V_R
                self.instances[self.instance_index].V_R += dV_R
            #hopefully do KAIN here
            
            #check convergence
            update = dV_R.norm()
            E_r = self.computeEnergy()
            dE_r = E_r - E_r_old
            E_r_old = E_r
            print(f"{i} \t |{self.instances[self.instance_index].V_R.norm()}\t |{update} \t |{E_r} \t |{dE_r}")
           
            if (update < prec):
                break
        else:
            raise RuntimeError("Did not converge")
        
    def computeEnergy(self):
        return (1/2)*vp.dot(self.Density, self.instances[self.instance_index].V_R)
        


In [4]:
help(vp.FunctionMap)

Help on class FunctionMap in module vampyr._vampyr.vampyr3d:

class FunctionMap(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      FunctionMap
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(...)
 |      __call__(self: vampyr._vampyr.vampyr3d.FunctionMap, inp: mrcpp::FunctionTree<3>) -> mrcpp::FunctionTree<3>
 |  
 |  __init__(...)
 |      __init__(self: vampyr._vampyr.vampyr3d.FunctionMap, fmap: Callable[[float], float], prec: float) -> None
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from pybind11_builtins.pybind11_object:
 |  
 |  __new__(*args, **kwargs) from pybind11_builtins.pybind11_type
 |      Create and return a new object.  See help(type) for accurate signature.



In [5]:

class PBESolver(GPESolver):
    def __init__(self, rho, eps, kappa, Poisson_operator, Derivative_operator):
        super().__init__(rho, eps, Poisson_operator, Derivative_operator)
        self.k_sq = kappa
        
    def computeGamma(self, V_tot):
        gamma = super().computeGamma(V_tot)
        global epsilon
        sinh = vp.FunctionMap(np.sinh, epsilon)
        
        return gamma - (1/(4*np.pi))*self.k_sq * sinh(V_tot)
        
        
        

In [6]:
class LPBESolver(PBESolver):
    def computeGamma(self, V_tot):
        gamma =  super(PBESolver, self).computeGamma(V_tot)
        return gamma - (1/(4*np.pi))*(self.k_sq * V_tot)

In [7]:
# Global parameters
k = 7                        # Polynomial order
L = [-20,20]                 # Simulation box size
epsilon = 1.0e-5             # Relative precision

# Define MRA and multiwavelet projector
MRA = vp.MultiResolutionAnalysis(order=k, box=L)
P_eps = vp.ScalingProjector(mra=MRA, prec=epsilon)
D_abgv = vp.ABGVDerivative(mra=MRA, a=0.0, b=0.0)
Poissop = vp.PoissonOperator(mra=MRA, prec=epsilon)

coords = [[0.0000000000,    0.0000000000,    0.000000000]] #centered in 
radii = [3.7794522492515403]  

width = 0.2

C = Cavity(coords, radii, width)
lin_perm = LinPermittivity(C, inside=1.0, outside=80.0)
exp_perm = ExpPermittivity(C, inside=1.0, outside=80.0)
kappa_sq = DHScreeningSq(C, inside=0.0, outside=compute_kappa_out(80.0, 0.1))


lin_perm_tree = P_eps(lin_perm)
exp_perm_tree = P_eps(exp_perm)
k_sq_tree = P_eps(kappa_sq)

grad_perm =vp.gradient(oper=D_abgv, inp=lin_perm_tree)
# nuclear density and total molecular density to compute the vacuum potential
beta = 10000
alpha = (beta / np.pi)**(3.0/2.0)
charge = 1.0
dens = P_eps(vp.GaussFunc(beta=beta, alpha=alpha*charge, position=[0.0, 0.0, 0.0], poly_exponent=[0,0,0]))

In [8]:
SCRF = GPESolver(dens, lin_perm_tree, Poissop, D_abgv)
reaction_op = Reaction_operator(SCRF)
reaction_op.setup(epsilon)


print("E_R: ", reaction_op.trace())

Initializing reaction operator
Setting up reaction operator
iter	|V_R norm		|update		|E_r		|E_r update
0 	 |0.46153808453951933	 |0.46153808453951933 	 |-0.0038379590376876166 	 |-0.0038379590376876166
1 	 |0.38117278597108	 |0.0803666663099235 	 |-0.003185953864909479 	 |0.0006520051727781375
2 	 |0.3905074914727334	 |0.009335099732153653 	 |-0.0032606573486507833 	 |-7.470348374130414e-05
3 	 |0.3896939738350803	 |0.0008135671077001632 	 |-0.003254294259843214 	 |6.363088807569486e-06
4 	 |0.38975070538392714	 |5.6735965113054705e-05 	 |-0.0032547439036278943 	 |-4.496437846805354e-07
5 	 |0.38974741205228675	 |3.293637213413585e-06 	 |-0.0032547179270146446 	 |2.5976613249757002e-08
E_R:  -0.0032547179270146446


In [9]:
PB_SCRF = PBESolver(dens, lin_perm_tree, k_sq_tree, Poissop, D_abgv)
reaction_op = Reaction_operator(PB_SCRF)
reaction_op.setup(epsilon)


print("E_R: ", reaction_op.trace())

Initializing reaction operator
Setting up reaction operator
iter	|V_R norm		|update		|E_r		|E_r update
0 	 |0.5283853925310578	 |0.5283853925310578 	 |-0.004032128059955803 	 |-0.004032128059955803
1 	 |0.42082700661227	 |0.10769726211526977 	 |-0.0033015938352038443 	 |0.0007305342247519585
2 	 |0.4357856859705138	 |0.015031983112652065 	 |-0.003392373824386607 	 |-9.077998918276264e-05
3 	 |0.4341771550504315	 |0.0016242418480586335 	 |-0.0033837223602735922 	 |8.651464113014658e-06
4 	 |0.4343206269837735	 |0.0001457924971368865 	 |-0.003384413046445208 	 |-6.906861716158598e-07
5 	 |0.43430948691903215	 |1.1400457496833228e-05 	 |-0.0033843654475593007 	 |4.759888590742667e-08
6 	 |0.434310267550731	 |8.0463206762372e-07 	 |-0.0033843684201579037 	 |-2.972598603032117e-09
E_R:  -0.0033843684201579037


In [10]:
LPBE_SCRF = LPBESolver(dens, lin_perm_tree,k_sq_tree, Poissop, D_abgv)
reaction_op = Reaction_operator(LPBE_SCRF)
reaction_op.setup(epsilon)


print("E_R: ", reaction_op.trace())

Initializing reaction operator
Setting up reaction operator
iter	|V_R norm		|update		|E_r		|E_r update
0 	 |0.5283848691683786	 |0.5283848691683786 	 |-0.0040321258432253065 	 |-0.0040321258432253065
1 	 |0.42082691709615144	 |0.10769682551395736 	 |-0.003301593401792444 	 |0.0007305324414328624
2 	 |0.43578552101622725	 |0.01503190649735109 	 |-0.003392373107477887 	 |-9.077970568544284e-05
3 	 |0.4341770020363683	 |0.0016242296623766403 	 |-0.0033837216868463395 	 |8.651420631547533e-06
4 	 |0.43432047250128086	 |0.0001457909898584422 	 |-0.003384412367924373 	 |-6.906810780336198e-07
5 	 |0.4343093325889839	 |1.1400300259207605e-05 	 |-0.003384364769542229 	 |4.7598382144162926e-08
6 	 |0.43431011320676977	 |8.046176590384838e-07 	 |-0.003384367742096392 	 |-2.9725541628861096e-09
E_R:  -0.003384367742096392
